In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- Parameters ---
L = 1.0          # Length of the domain
Nx = 100         # Number of spatial steps (101 grid points)
dx = L / Nx      # Spatial step size (0.01 as requested)
dt = 0.005       # Time step size (0.005 as requested)
c = 1.0          # Wave speed (assumed to be 1 for a stable simulation with given dt, dx)
Cr = c * dt / dx # Courant number (Cr = 0.5)

# Verify step size
if abs(dx - 0.01) > 1e-6 or abs(dt - 0.005) > 1e-6:
    print(f"Warning: dx is {dx}, dt is {dt}. Requested values were 0.01 and 0.005.")
if Cr > 1:
    print(f"Error: Courant number {Cr} exceeds stability limit (Cr <= 1).")
    exit()

# --- Grid Initialization ---
x = np.linspace(0, L, Nx + 1)
# Three arrays to store solution at t-dt (u_prev), t (u_curr), t+dt (u_next)
u_prev = np.zeros(Nx + 1)
u_curr = np.zeros(Nx + 1)
u_next = np.zeros(Nx + 1)

# --- Initial Conditions ---
# Initial wave shape: u(x, 0) = e^(-100 * (x - 0.5)^2)
for i in range(Nx + 1):
    u_curr[i] = np.exp(-100 * (x[i] - 0.5)**2)

# Initial time derivative: (du/dt)(x, 0) = 0
# For the standard FDTD scheme, this requires a special starting condition for the first time step
# u^1_i = u^0_i + dt * u_t^0_i + 0.5 * dt^2 * u_tt^0_i
# Since u_t = 0, u^1_i approx u^0_i + 0.5 * dt^2 * c^2 * u_xx^0_i
# We approximate u^1_i using a central difference for the second spatial derivative
for i in range(1, Nx):
    u_prev[i] = u_curr[i] + 0.5 * Cr**2 * (u_curr[i-1] - 2 * u_curr[i] + u_curr[i+1])
# Boundary conditions apply to u_prev at i=0 and i=Nx (already 0)

# --- Time stepping loop (Example for a few steps) ---
num_time_steps = 200 # Total simulation time T = num_time_steps * dt

for n in range(num_time_steps):
    # Apply standard FDTD update rule for inner points (1 to Nx-1)
    for i in range(1, Nx):
        u_next[i] = -u_prev[i] + 2 * u_curr[i] + Cr**2 * (u_curr[i-1] - 2 * u_curr[i] + u_curr[i+1])

    # Apply Boundary Conditions: u(0, t) = 0, u(L, t) = 0
    u_next[0] = 0
    u_next[Nx] = 0

    # Update solution arrays for the next time step
    u_prev = u_curr.copy()
    u_curr = u_next.copy()

    # Optional: Add plotting/visualization code here to see the wave propagate
    if n % 50 == 0:
        plt.plot(x, u_curr, label=f't={n*dt:.2f}')
        plt.title('Wave Propagation')
        plt.xlabel('x')
        plt.ylabel('u(x,t)')
        plt.legend()
        plt.grid(True)
        # plt.show() # Use this if running in an environment that displays plots immediately
